In [ ]:
Tạo đường dẫn đến datasets

In [4]:
!mkdir -p ~/.kaggle
!cp /kaggle/input/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat '/kaggle/input/kaggle/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
!ls -la ~/.kaggle


total 12
drwxr-xr-x 2 root root 4096 Dec 11 08:57 .
drwx------ 1 root root 4096 Dec 11 08:57 ..


In [5]:
!kaggle datasets list | head

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


**Bắt đầu train**

In [2]:
!pip cache purge
!pip install --upgrade pip
!pip install --upgrade setuptools wheel

Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.6 MB/s  0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is incompatible.


In [3]:
!pip uninstall -y numpy ultralytics opencv-python opencv-python-headless
!rm -rf /usr/local/lib/python3.11/dist-packages/numpy*
!rm -rf /usr/local/lib/python3.11/dist-packages/ultralytics*
!rm -rf /usr/local/lib/python3.11/dist-packages/cv2*

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88


In [ ]:
!pip install roboflow

In [4]:
!pip uninstall -y numpy ultralytics opencv-python-headless
!pip install numpy==1.26.4 --no-cache-dir
!pip install ultralytics==8.3.220 opencv-python-headless==4.10.0.84 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 197.5 MB/s  0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires opencv-python-headless, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have p

Kiểm tra version YOLO và GPU

In [5]:
import numpy, ultralytics, torch, cv2
print("numpy:", numpy.__version__)
print("ultralytics:", ultralytics.__version__)
print("torch:", torch.__version__)
print("cv2:", cv2.__version__)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
numpy: 1.26.4
ultralytics: 8.3.220
torch: 2.6.0+cu124
cv2: 4.10.0


In [6]:
import ultralytics
print(ultralytics.__version__)


8.3.220


In [7]:
from ultralytics import YOLO
model = YOLO("/kaggle/input/dataset/yolo12n.pt");

In [ ]:
#Nếu sử dụng dataset từ Roboflow
!pip install roboflow --quiet
from roboflow import Roboflow

rf = Roboflow(api_key="YOUR_API_KEY_HERE")
project = rf.workspace("your-workspace-name").project("your-project-name")
dataset = project.version(1).download("yolov12")  # hoặc "yolov12" nếu bạn dùng YOLOv12


PRE-TRAIN nếu chưa có file pretrain.pt (build model từ đầu)

In [10]:
import yaml

# 1️⃣ Đọc file gốc (read-only)
with open("/kaggle/working/multi_dataset.yaml", "r", encoding="utf-8") as f:
    raw = f.read()

print("🧐 Original content:")
print(raw[:300])  # xem thử vài dòng đầu


🧐 Original content:
path: /kaggle/input/licenceplate
train: train/images
val: valid/images
names:
  0: License_Plate



In [9]:
import yaml

yaml_content = {
    'path': '/kaggle/input/licenceplate',
    'train': 'train/images',
    'val': 'valid/images',
    'names': {
        0: 'License_Plate',
    }
}

with open("/kaggle/working/multi_dataset.yaml", "w", encoding="utf-8") as f:
    yaml.dump(yaml_content, f, sort_keys=False, allow_unicode=True)

print("✅ File YAML đã được tạo tại /kaggle/working/multi_dataset.yaml")

# Kiểm tra load lại
with open("/kaggle/working/multi_dataset.yaml", "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)
    print("✅ YAML loaded successfully!")
    print(data)


✅ File YAML đã được tạo tại /kaggle/working/multi_dataset.yaml
✅ YAML loaded successfully!
{'path': '/kaggle/input/licenceplate', 'train': 'train/images', 'val': 'valid/images', 'names': {0: 'License_Plate'}}


In [11]:
###Using Kaggle notes to train instead of on terminal
from ultralytics import YOLO
import os, shutil

# Đường dẫn thư mục "runs"
runs_dir = "/kaggle/working/"
train_dir = os.path.join(runs_dir, "train")

# Xoá thư mục train cũ (nếu có)
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
    print(f"Deleted folder: {train_dir}")

# Main
if __name__ == "__main__":
    # Load model YOLOv12
    model = YOLO("yolo12n.pt")

    # Train
    results = model.train(
        data="/kaggle/working/multi_dataset.yaml",  #link to dataset yaml
        epochs=100,
        imgsz=640,      #Giảm về 512 nếu train trực tiếp trên laptop
        batch=8,        # có thể tăng batch size vì GPU Colab mạnh hơn
        workers=2,
        device=0,
        cache=True
    )

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.220 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/multi_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=No

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1877       1947      0.983      0.974      0.988      0.724
Speed: 0.2ms preprocess, 3.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/runs/detect/train


In [12]:
import os

path = "/kaggle/working/runs/detect/"
print("Files in runs folder:", os.listdir(path))

Files in runs folder: ['train']


In [13]:
!zip -r /kaggle/working/trained_model.zip /kaggle/working/runs/detect/train

  adding: kaggle/working/runs/detect/train/ (stored 0%)
  adding: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/train/train_batch164072.jpg (deflated 13%)
  adding: kaggle/working/runs/detect/train/confusion_matrix.png (deflated 34%)
  adding: kaggle/working/runs/detect/train/results.csv (deflated 64%)
  adding: kaggle/working/runs/detect/train/BoxF1_curve.png (deflated 16%)
  adding: kaggle/working/runs/detect/train/val_batch2_labels.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/train/val_batch2_pred.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/train/val_batch0_labels.jpg (deflated 7%)
  adding: kaggle/working/runs/detect/train/results.png (deflated 8%)
  adding: kaggle/working/runs/detect/train/BoxPR_curve.png (deflated 26%)
  adding: kaggle/working/runs/detect/train/labels.jpg (deflated 45%)
  adding: kaggle/working/runs/detect/train/BoxR_curve.png (deflated 16%)
  adding: kaggle/working/runs/detect/train/val_

Resume training nếu đã có sẵn model last.pt từ trước

In [ ]:
model = YOLO("/kaggle/input/resume/last.pt") #link to 'last.pt'

model.train(
    data="/kaggle/working/multi_dataset.yaml",
    epochs=50,       # train thêm 50 epoch nữa
    imgsz=640,
    batch=8,         #Batch = 8 is use GPU T4 on Colab
    workers=2,
    device=0,
)

TEST

In [ ]:
!pip install pillow
import os
from ultralytics import YOLO
from IPython.display import Image, display
display(Image(filename="/kaggle/input/tesstt/image_2025-10-26_181752780.png"))


#Load trained model YOLOv12 after epoched - choose the best.pt
model = YOLO(r"/kaggle/working/runs/detect/train3/weights/best.pt")

result = model(Image)

#Print the result to screen
for r in result:
    print(r.boxes)
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1]) #RGB PIL image
    im.show()
    #save_path = os.path.join('D:\VSCode\DCLP\main_code/result/vehicle_detect', 'result.jpg')
    #im.save(save_path)

